<a href="https://colab.research.google.com/github/setthawut8/ai/blob/main/modified/%5BEN2THDict4Vader%5D_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inspiration: https://github.com/cjhutto/vaderSentiment

#Library

In [ ]:
!pip install fairseq
!pip install pythainlp
!pip install sacremoses
!pip install transformers
!pip install vaderSentiment
!git clone https://github.com/cjhutto/vaderSentiment.git

In [18]:
import nltk
nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


#Methods for the Vader Modification

In [3]:
#Change the vader lexicon
#https://stackoverflow.com/questions/45275166/is-vader-sentimentintensityanalyzer-multilingual#:~:text=Change%20the-,vader_lexicon,-.txt
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
#How to add or delete new word(s) in the Vader Lexicon as the link below:
#https://stackoverflow.com/questions/40481348/is-it-possible-to-edit-nltks-vader-sentiment-lexicon#:~:text=in%20this%20post%3A-,from%20nltk.sentiment.vader%20import%20SentimentIntensityAnalyzer,-new_words%20%3D%20%7B%0A%20%20%20%20%27foo

# from nltk.sentiment.vader import SentimentIntensityAnalyzer

# new_words = {
#     'foo': 2.0,
#     'bar': -3.4,
# }

##Add some words
# SIA = SentimentIntensityAnalyzer()
# SIA.lexicon.update(new_words)

## Delete some words
# SIA = SentimentIntensityAnalyzer()
# SIA.lexicon.pop('no')

#Vader EN Text

In [4]:
NEGATE = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

# (empirically derived mean sentiment intensity rating increase for booster words)
B_INCR = 0.293
B_DECR = -0.293

# (empirically derived mean sentiment intensity rating increase for using ALLCAPs to emphasize a word)
C_INCR = 0.733
N_SCALAR = -0.74

BOOSTER_DICT = {"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR,
     "completely": B_INCR, "considerable": B_INCR, "considerably": B_INCR,
     "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormous": B_INCR, "enormously": B_INCR,
     "entirely": B_INCR, "especially": B_INCR, "exceptional": B_INCR, "exceptionally": B_INCR,
     "extreme": B_INCR, "extremely": B_INCR,
     "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR, "frackin": B_INCR, "fracking": B_INCR,
     "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR,
     "fuckin": B_INCR, "fucking": B_INCR, "fuggin": B_INCR, "fugging": B_INCR,
     "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR,
     "incredible": B_INCR, "incredibly": B_INCR, "intensely": B_INCR,
     "major": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
     "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
     "so": B_INCR, "substantially": B_INCR,
     "thoroughly": B_INCR, "total": B_INCR, "totally": B_INCR, "tremendous": B_INCR, "tremendously": B_INCR,
     "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utter": B_INCR, "utterly": B_INCR,
     "very": B_INCR,
     "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
     "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
     "less": B_DECR, "little": B_DECR, "marginal": B_DECR, "marginally": B_DECR,
     "occasional": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
     "scarce": B_DECR, "scarcely": B_DECR, "slight": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
     "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}

# check for sentiment laden idioms that do not contain lexicon words (future work, not yet implemented)
SENTIMENT_LADEN_IDIOMS = {"cut the mustard": 2, "hand to mouth": -2,
                          "back handed": -2, "blow smoke": -2, "blowing smoke": -2,
                          "upper hand": 1, "break a leg": 2,
                          "cooking with gas": 2, "in the black": 2, "in the red": -2,
                          "on the ball": 2, "under the weather": -2}

# check for special case idioms and phrases containing lexicon words
SPECIAL_CASES = {"the shit": 3, "the bomb": 3, "bad ass": 1.5, "badass": 1.5, "bus stop": 0.0,
                 "yeah right": -2, "kiss of death": -1.5, "to die for": 3,
                 "beating heart": 3.1, "broken heart": -2.9 }


#EN2TH Translation

In [ ]:
#Translation
from pythainlp.translate import Translate
en2th = Translate('en', 'th')

##In The PY File

In [6]:
NEGATE_translated = [en2th.translate(x) for x in NEGATE]
NEGATE_translated_modified = [x for x in NEGATE_translated if x != 'Name=สมุดที่อยู่ - KName']
print(NEGATE_translated_modified)

['ไม่ใช่นะ', 'ไม่อ่ะ', 'ไม่สามารถ', 'ไม่ได้อ่ะ', 'ไม่สามารถ', 'ไม่อ่ะ', 'ไม่ได้', 'ไม่ได้', 'ไม่ใช่', 'ไม่ได้', 'ทําไม่ได้', 'ไม่กล้าไม่กล้า', 'ไม่ได้', 'ไม่ได้', 'ไม่อ่ะ', 'ไม่อ่ะ', 'ยังไม่ได้', 'ไม่ใช่', 'ไม่เหมือนกัน', 'อย่านะ', 'ไม่ได้', 'ยังเลย', 'ยังเลย', 'ไม่ใช่', 'ไม่ได้', 'อย่าเลย', 'ไม่จําเป็น', 'ไม่เคย', 'ไม่มี', 'ไม่อ่ะ', 'หรือ', 'ไม่ใช่', 'ไม่มีอะไร', 'ไม่มีที่ไหนเลย', 'ไม่น่าจะ', 'ไม่ใช่', 'ไม่', 'ไม่ควรจะ', 'ชันไม่', 'ไม่ควร', 'ไม่ใช่', 'ไม่ได้', 'โดยไม่ต้อง', 'ไม่อ่ะ', 'ไม่อยากหรอก', 'จะไม่', 'จะไม่', 'ไม่ค่อย', 'ไม่ค่อยได้', 'แม้จะ']


In [7]:
BOOSTER_DICT_modified = {en2th.translate(k): v for k, v in BOOSTER_DICT.items()}
print(BOOSTER_DICT_modified)

{'แน่นอนครับ': 0.293, 'น่าอัศจรรย์': 0.293, 'น่ากลัวจัง': 0.293, 'สมบูรณ์': 0.293, 'มาก': 0.293, 'เด็ด': 0.293, 'ลึก': 0.293, 'ไหล...': 0.293, 'ใหญ่มาก': 0.293, 'อย่างมหาศาล': 0.293, 'ทั้งหมด': 0.293, 'โดยเฉพาะอย่างยิ่ง': 0.293, 'พิเศษ': 0.293, 'พิเศษสุด': 0.293, 'สุดขั้ว': 0.293, 'สุด ๆ': 0.293, 'สวยจริงๆ': 0.293, 'พลิกภาพ': 0.293, 'flippin': 0.293, 'Name=สมุดที่อยู่ - KName': -0.293, 'การไล่ล่า': 0.293, 'ไอ้งั่ง': 0.293, 'ไอ้บ้าเอ๊ย': 0.293, 'ไอ้บ้าเอ้ย': 0.293, 'เต็มที่': 0.293, 'ร่วมเพศ': 0.293, 'อย่างมาก': 0.293, 'สวัสดี': 0.293, 'อย่างสูง': 0.293, 'เหลือเชื่อ': 0.293, 'ไม่น่าเชื่อเลย': 0.293, 'หลัก@ item: inlistbox': 0.293, 'เฉดเดียวกัน': 0.293, 'มากขึ้น': 0.293, 'มากที่สุด': 0.293, 'ล้วน ๆ': 0.293, 'ค่อนข้าง': -0.293, 'จริงๆเลย': 0.293, 'น่าทึ่งมาก': 0.293, 'ดังนั้น': 0.293, 'อย่างทั่วถึง': 0.293, 'รวมทั้งหมด': 0.293, 'โดยสิ้นเชิง': 0.293, 'อูเบอร์ค่ะ': 0.293, 'ผิดปกติ': 0.293, 'ที่สุด': 0.293, 'อย่างที่สุด': 0.293, 'เกือบแล้ว': -0.293, 'แทบจะไม่': -0.293, 'พอแล้ว': -0.293, 'ประ

In [8]:
SENTIMENT_LADEN_IDIOMS_modified = {en2th.translate(k): v for k, v in SENTIMENT_LADEN_IDIOMS.items()}
print(SENTIMENT_LADEN_IDIOMS_modified)

{'ตัดมัสตาร์ด': 2, 'มือต่อปาก': -2, 'กลับมือ': -2, 'ระเบิดควัน': -2, 'มือบน': 1, 'หักขา': 2, 'ปรุงอาหารด้วยแก๊ส': 2, 'แบล็กค่ะ': 2, 'สีแดงจ้ะ': -2, 'บนลูกบอล': 2, 'ภายใต้สภาพอากาศ': -2}


In [9]:
SPECIAL_CASES_modified = {en2th.translate(k): v for k, v in SPECIAL_CASES.items()}
print(SPECIAL_CASES_modified)

{'อึ': 3, 'ระเบิด': 3, 'ตูดไม่ดี': 1.5, 'ตัวแสบ': 1.5, 'ป้ายรถเมล์': 0.0, 'ใช่ ถูกต้อง': -2, 'จูบแห่งความตาย': -1.5, 'ตายเพื่อ': 3, 'หัวใจเต้น': 3.1, 'หัวใจสลาย': -2.9}


## In The TXT File

In [74]:
#load data from the txt
def open_txt_file(txt_path):
  file = open(txt_path, 'r')
  text = file.read()
  file.close()
  return text

data = open_txt_file('/content/vaderSentiment/vaderSentiment/vader_lexicon.txt')

In [116]:
data[:50]

'$:\t-1.5\t0.80623\t[-1, -1, -1, -1, -3, -1, -3, -1, -2, -1]\n%)\t-0.4\t1.0198\t[-1, 0, -1, 0, 0, -2, -1, 2,'

In [76]:
#Create arrays for the data
data_lists = data.split('\n')
data_list_lists = [x.split('\t') for x in data_lists]

In [77]:
#Check if there are more than 4 for each word
count_more_than_four = 0

for text in data_list_lists:
  if len(text) != 4:
    count_more_than_four += 1
    print("The unequal to four index:", data_list_lists.index(text))
  continue
print("no. of the unequal to four:", count_more_than_four)

no. of the unequal to four: 0


In [78]:
#Check if there are more than one words
count_blank_spaces = 0
for text in data_list_lists:
  if ' ' in text[0]:
    count_blank_spaces += 1
    print("The blank space index:", data_list_lists.index(text))
  continue
print("no. of blank spaces:", count_blank_spaces)

The blank space index: 5
The blank space index: 1260
The blank space index: 2865
The blank space index: 5908
no. of blank spaces: 4


In [79]:
blank_space_lists = [5, 1260, 2865, 5908]

for idx in blank_space_lists:
  print(data_list_lists[idx])

["( '}{' )", '1.6', '0.66332', '[1, 2, 2, 1, 1, 2, 2, 1, 3, 1]']
["can't stand", '-2.0', '0.63246', '[-2, -2, -2, -1, -1, -2, -3, -2, -2, -3]']
['fed up', '-1.8', '1.249', '[-2, -2, -3, -4, -2, -2, -1, -2, -1, 1]']
['screwed up', '-1.5', '0.67082', '[-2, -2, -2, -1, -1, 0, -2, -1, -2, -2]']


In [80]:
# #Method to check as the word is English.
# import nltk
# nltk.download('words')
# from nltk.corpus import words

# samplewords=['apple','a%32','j & quod','rectangle','house','fsdfdsoij','fdfd']
# [i for i in samplewords if i in words.words()]

In [81]:
index_lists = [5, 1260, 2865, 5908]

for idx in index_lists:
  sample_data_in_lists = data_list_lists[idx][0]
  print("Text :", sample_data_in_lists)
  print("Is English: ", sample_data_in_lists in words.words())
  print("==================")

Text : ( '}{' )
Is English:  False
Text : can't stand
Is English:  False
Text : fed up
Is English:  False
Text : screwed up
Is English:  False


In [82]:
th_text_sentiments = []
en_sentiment_index = []
count_translated = 0
count_all_texts = 0
for text in data_list_lists:
  count_all_texts += 1
  #If english, translate
  if text[0] in words.words():
    en_sentiment_index.append(data_list_lists.index(text))
    th_texts = en2th.translate(text[0])
    new_th_sentiments = [th_texts, text[1], text[2], text[3]]
    th_text_sentiments.append(new_th_sentiments)
    count_translated += 1
  else:
    continue

In [107]:
## Delete error on translation
#Name=สมุดที่อยู่ - KName error in translation in PyThaiNLP
#@ item: inlistbox
th_sentiment_index_delelted = []
for arr in th_text_sentiments:
  if 'Name=สมุดที่อยู่' in arr[0]:
    th_sentiment_index_delelted.append(th_text_sentiments.index(arr))
    th_text_sentiments.remove(arr)

  elif '@ item: inlistbox' in arr[0]:
    th_sentiment_index_delelted.append(th_text_sentiments.index(arr))
    th_text_sentiments.remove(arr)

  elif 'Name=สมุดที่อยู่ - KName' in arr[0]:
    th_sentiment_index_delelted.append(th_text_sentiments.index(arr))
    th_text_sentiments.remove(arr)  

  elif '@ info: whatsthis' in arr[0]:
    th_sentiment_index_delelted.append(th_text_sentiments.index(arr))
    th_text_sentiments.remove(arr)
  else:
    continue
#9558

In [108]:
print(len(data_list_lists), count_all_texts, len(th_text_sentiments), count_translated)

7520 7520 2651 3997


In [109]:
th_text_sentiments[:5]

[['ฟู', '-3.7', '0.45826', '[-3, -4, -4, -3, -3, -4, -4, -4, -4, -4]'],
 ['เทฮัก', '1.9', '0.7', '[3, 1, 2, 2, 1, 2, 3, 2, 1, 2]'],
 ['มือ', '2.2', '0.87178', '[2, 2, 1, 3, 2, 3, 4, 1, 2, 2]'],
 ['หัวใจ', '3.2', '0.63246', '[3, 3, 4, 3, 4, 2, 3, 4, 3, 3]'],
 ['หัวใจ', '3.3', '0.48305', '[4, 3, 3, 3, 4, 3, 3, 3, 4, 3]']]

In [110]:
check_en_sentiment = index[:5]
for idx in check_en_sentiment:
  print(data_list_lists[idx])

['(-:{', '-0.1', '1.57797', '[-2, -3, 1, -2, 1, 1, 0, 0, 2, 1]']
['(;<', '0.3', '1.00499', '[1, 2, -1, -1, 0, 0, 1, -1, 1, 1]']
['(^;', '1.5', '0.5', '[1, 2, 2, 1, 2, 1, 2, 1, 1, 2]']
['(^;o', '1.9', '0.83066', '[2, 2, 1, 2, 1, 4, 2, 1, 2, 2]']
['*-;', '2.4', '1.62481', '[2, 3, 4, 4, 2, 1, -1, 4, 1, 4]']


#Change Arrays into String

In [111]:
th_text_sentiments[0]

['ฟู', '-3.7', '0.45826', '[-3, -4, -4, -3, -3, -4, -4, -4, -4, -4]']

In [112]:
#\t for one blank space, \n for the new spaces
join_with_blank_spaces = ['\t'.join(text) for text in th_text_sentiments]
final_th_text_with_sentiment = '\n'.join(join_with_blank_spaces)

In [118]:
final_th_text_with_sentiment[:50]

'ฟู\t-3.7\t0.45826\t[-3, -4, -4, -3, -3, -4, -4, -4, -'

#Add translated to the txt file

In [115]:
#Inspiration: https://colab.research.google.com/github/computationalcore/introduction-to-python/blob/master/notebooks/4-files/PY0101EN-4-2-WriteFile.ipynb#scrollTo=Ar3kQmbt0uq3

lexicon_path = '/content/vaderSentiment/vaderSentiment/vader_lexicon.txt'
with open(lexicon_path, 'a') as new_lexicon:
  new_lexicon.write(final_th_text_with_sentiment)